In [ ]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import numpy as np
from itertools import combinations
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

# Suppress the warning for deprecated frame.append
pd.set_option('mode.chained_assignment', None)



In [ ]:
#Loading and renaming datasets for easy use. Qualitative is named as store_features
timeseries = pd.read_csv('/content/gdrive/MyDrive/Final Capstone - MSBA/Maverik/time_series_data_msba.csv')
store_features = pd.read_csv('/content/gdrive/MyDrive/Final Capstone - MSBA/Maverik/qualitative_data_msba.csv')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Process timeseries data
timeseries['capital_projects.soft_opening_date'] = pd.to_datetime(timeseries['capital_projects.soft_opening_date'])
timeseries['calendar.calendar_day_date'] = pd.to_datetime(timeseries['calendar.calendar_day_date'])

timeseries = timeseries.set_index('calendar.calendar_day_date')

timeseries['Year'] = timeseries.index.year
timeseries['Month'] = timeseries.index.month
timeseries['Day'] = timeseries.index.day
timeseries['DayOfYear'] = timeseries.index.dayofyear

def map_month_to_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

timeseries['season'] = timeseries.index.month.map(map_month_to_season)
timeseries.sort_index(inplace=True)
timeseries.dropna(inplace=True)

# One-hot encode categorical columns
categorical_cols = ['calendar.day_of_week', 'calendar_information.holiday', 'calendar_information.type_of_day', 'season']
timeseries = pd.get_dummies(timeseries, columns=categorical_cols)
print(timeseries.head())

# Process store features data
categorical_columns = ['lottery', 'freal', 'bonfire_grill', 'pizza', 'cinnabon', 'godfather_s_pizza',
                       'ethanol_free', 'diesel', 'hi_flow_lanes', 'rv_lanes', 'hi_flow_rv_lanes', 'def',
                       'cat_scales', 'car_wash', 'ev_charging', 'rv_dumps', 'propane',
                       'traditional_forecourt_layout', 'traditional_forecourt_stack_type',
                       'rv_lanes_layout', 'rv_lanes_stack_type', 'hi_flow_lanes_layout',
                       'hi_flow_lanes_stack_type', 'hi_flow_rv_lanes_layout', 'hi_flow_rv_lanes_stack_type',
                       'non_24_hour', 'self_check_out']

store_features = pd.get_dummies(store_features, columns=categorical_columns)
store_features.fillna(0, inplace=True)

# Scale numerical columns
numerical_columns = ['open_year', 'square_feet', 'front_door_count', 'years_since_last_project', 'parking_spaces',
                     'x1_mile_pop', 'x1_mile_emp', 'x1_mile_income', 'x1_2_mile_pop', 'x1_2_mile_emp',
                     'x1_2_mile_income', 'x5_min_pop', 'x5_min_emp', 'x5_min_inc', 'x7_min_pop', 'x7_min_emp',
                     'x7_min_inc', 'traditional_forecourt_fueling_positions', 'hi_flow_lanes_fueling_positions',
                     'hi_flow_lanes_fueling_positions_2', 'rv_lanes_fueling_positions', 'rv_lanes_fueling_positions_2',
                     'mens_toilet_count', 'mens_urinal_count', 'womens_toilet_count', 'womens_sink_count']

scaler = StandardScaler()
store_features[numerical_columns] = scaler.fit_transform(store_features[numerical_columns])
store_features.fillna(0, inplace=True)

# List of sales metrics
sales_metrics = [
    'daily_yoy_ndt.total_inside_sales',
    'daily_yoy_ndt.total_food_service',
    'diesel',
    'unleaded'
]


                            Unnamed: 0 capital_projects.soft_opening_date  \
calendar.calendar_day_date                                                  
2021-01-12                       13814                         2021-01-12   
2021-01-13                       13815                         2021-01-12   
2021-01-14                       13816                         2021-01-12   
2021-01-15                       13726                         2021-01-12   
2021-01-16                       13817                         2021-01-12   

                            calendar.fiscal_week_id_for_year  \
calendar.calendar_day_date                                     
2021-01-12                                                 2   
2021-01-13                                                 2   
2021-01-14                                                 2   
2021-01-15                                                 3   
2021-01-16                                                 3   

           

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

def xgboost_forecast(timeseries, store_features, sales_metrics, forecast_days=365):
    all_forecasts = {}

    for metric in sales_metrics:
        site_forecasts = {}

        for site_id in timeseries['site_id_msba'].unique():
            # Filter data for a specific site
            site_data = timeseries[timeseries['site_id_msba'] == site_id]

            # Merge site-specific features from store_features
            site_features = store_features[store_features['site_id_msba'] == site_id]
            site_data = pd.merge(site_data, site_features, on='site_id_msba', how='left')

            # Features and target variable
            X = site_data.drop(columns=[metric])  # All columns except the target
            y = site_data[metric]

            # Drop non-numeric columns
            non_numeric_cols = X.select_dtypes(exclude=['int', 'float']).columns
            X = X.drop(columns=non_numeric_cols)

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

            # Initialize and fit XGBoost model
            model = XGBRegressor(n_estimators=100, learning_rate=0.1)  # Modify hyperparameters
            model.fit(X_train, y_train)

            # Make predictions for the test dataset
            test_forecast = model.predict(X_test)

            # Extend the test dataset for forecasting
            X_future = X_test.copy()
            future_forecasts = []
            for i in range(forecast_days):
                # Predict using the model for future days
                future_forecast = model.predict(X_future.iloc[[-1]])

                # Append forecast to the future forecasts
                future_forecasts.append(future_forecast[0])

                # Append the forecast to X_future for the next prediction
                X_future = X_future.append(X_future.iloc[[-1]], ignore_index=True)
                forecast_value = future_forecast[0].astype(np.float64)  # Convert to a suitable type, e.g., np.float64
                X_future.iloc[-1] = forecast_value

            # Ensure the length of future_forecasts matches forecast_days
            if len(future_forecasts) != forecast_days:
                raise ValueError("Forecast lengths are not consistent")

            # Store predictions for this site
            site_forecasts[site_id] = np.concatenate([test_forecast, future_forecasts])

            # Evaluate model (optional)
            mse = mean_squared_error(y_test, test_forecast)
            mae = mean_absolute_error(y_test, test_forecast)
            rmse = np.sqrt(mse)
            print(f"Site ID: {site_id} - Metric: {metric}, RMSE: {rmse:.2f}, MAE: {mae:.2f}")

        # Store forecasts for this metric across all sites
        all_forecasts[metric] = site_forecasts

    return all_forecasts


In [ ]:
# Example usage for multiple sales metrics with a forecast of 365 days
multiple_sales_forecasts_xgboost = xgboost_forecast(timeseries, store_features, sales_metrics, forecast_days=365)

Site ID: 21560 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 393.58, MAE: 222.87
Site ID: 22015 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 561.11, MAE: 423.61
Site ID: 22085 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 278.77, MAE: 213.21
Site ID: 22330 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 293.87, MAE: 220.18
Site ID: 22785 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 289.28, MAE: 228.04
Site ID: 22540 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 395.18, MAE: 339.19
Site ID: 22715 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 333.33, MAE: 291.59
Site ID: 22750 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 393.94, MAE: 292.41
Site ID: 23135 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 242.49, MAE: 186.45
Site ID: 22820 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 281.84, MAE: 218.82
Site ID: 22855 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 346.49, MAE: 255.21
Site ID: 22925 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

def xgboost_forecast(timeseries, store_features, sales_metrics, forecast_days=365):
    all_forecasts = {}

    for metric in sales_metrics:
        site_forecasts = {}

        for site_id in timeseries['site_id_msba'].unique():
            try:
                # Filter data for a specific site
                site_data = timeseries[timeseries['site_id_msba'] == site_id]

                # Merge site-specific features from store_features
                site_features = store_features[store_features['site_id_msba'] == site_id]
                site_data = pd.merge(site_data, site_features, on='site_id_msba', how='left')

                # Features and target variable
                X = site_data.drop(columns=[metric])  # All columns except the target
                y = site_data[metric]

                # Drop non-numeric columns
                non_numeric_cols = X.select_dtypes(exclude=['int', 'float']).columns
                X = X.drop(columns=non_numeric_cols)

                # Train-test split
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

                # Initialize and fit XGBoost model
                model = XGBRegressor(n_estimators=100, learning_rate=0.1)  # Modify hyperparameters
                model.fit(X_train, y_train)

                # Make predictions for the test dataset
                test_forecast = model.predict(X_test)

                # Extend the test dataset for forecasting
                X_future = X_test.copy()
                future_forecasts = []
                for i in range(forecast_days):
                    # Predict using the model for future days
                    future_forecast = model.predict(X_future.iloc[[-1]])

                    # Append forecast to the future forecasts
                    future_forecasts.append(future_forecast[0])

                    # Append the forecast to X_future for the next prediction
                    X_future = X_future.append(X_future.iloc[[-1]], ignore_index=True)
                    forecast_value = future_forecast[0].astype(np.float64)
                    X_future.iloc[-1] = forecast_value

                # Ensure the length of future_forecasts matches forecast_days
                if len(future_forecasts) != forecast_days:
                    raise ValueError("Forecast lengths are not consistent")

                # Store predictions for this site
                site_forecasts[site_id] = np.concatenate([test_forecast, future_forecasts])

                # Evaluate model (optional)
                mse = mean_squared_error(y_test, test_forecast)
                mae = mean_absolute_error(y_test, test_forecast)
                rmse = np.sqrt(mse)
                print(f"Site ID: {site_id} - Metric: {metric}, RMSE: {rmse:.2f}, MAE: {mae:.2f}")

            except Exception as e:
                print(f"Error occurred for Site ID: {site_id} - Metric: {metric}. Details: {str(e)}")

        # Store forecasts for this metric across all sites
        all_forecasts[metric] = site_forecasts

    return all_forecasts

# Example usage for multiple sales metrics with a forecast of 365 days
multiple_sales_forecasts_xgboost = xgboost_forecast(timeseries, store_features, sales_metrics, forecast_days=365)


Site ID: 21560 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 393.58, MAE: 222.87
Site ID: 22015 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 561.11, MAE: 423.61
Site ID: 22085 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 278.77, MAE: 213.21
Site ID: 22330 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 293.87, MAE: 220.18
Site ID: 22785 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 289.28, MAE: 228.04
Site ID: 22540 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 395.18, MAE: 339.19
Site ID: 22715 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 333.33, MAE: 291.59
Site ID: 22750 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 393.94, MAE: 292.41
Site ID: 23135 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 242.49, MAE: 186.45
Site ID: 22820 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 281.84, MAE: 218.82
Site ID: 22855 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 346.49, MAE: 255.21
Site ID: 22925 - Metric: daily_yoy_ndt.total_inside_sales, RMSE: 

In [ ]:
import numpy as np

# Provided RMSE values for 'daily_yoy_ndt.total_inside_sales'
rmse_values = [
    393.58, 561.11, 278.77, 293.87, 289.28, 395.18, 333.33, 393.94, 242.49, 281.84,
    346.49, 330.14, 491.20, 545.40, 432.48, 565.97, 254.68, 586.50, 273.53, 257.16,
    480.14, 304.73, 360.33, 302.97, 271.56, 296.62, 593.24, 320.07, 214.93, 294.30,
    314.13, 518.73, 384.22, 401.77, 528.17, 355.07, 203.32, 341.06
]

# Calculate cumulative RMSE
cumulative_rmse = np.sqrt(np.sum(np.square(rmse_values)))
print(f"Cumulative RMSE for 'daily_yoy_ndt.total_inside_sales' for all site ids: {cumulative_rmse:.2f}")


Cumulative RMSE for 'daily_yoy_ndt.total_inside_sales' for all site ids: 2374.15


In [ ]:
import numpy as np

# RMSE values for different sales metrics
sales_metrics = {
    'daily_yoy_ndt.total_inside_sales': [
        393.58, 561.11, 278.77, 293.87, 289.28, 395.18, 333.33, 393.94, 242.49, 281.84,
        346.49, 330.14, 491.20, 545.40, 432.48, 565.97, 254.68, 586.50, 273.53, 257.16,
        480.14, 304.73, 360.33, 302.97, 271.56, 296.62, 593.24, 320.07, 214.93, 294.30,
        314.13, 518.73, 384.22, 401.77, 528.17, 355.07, 203.32, 341.06
    ],
    'daily_yoy_ndt.total_food_service': [
        79.42, 105.85, 167.88, 111.29, 81.58, 110.56, 85.87, 102.87, 94.84, 55.49,
        82.83, 83.04, 86.92, 115.13, 76.22, 182.20, 158.37, 75.13, 78.87, 60.41,
        115.57, 171.95, 91.28, 95.50, 173.94, 98.89, 68.28, 125.92, 154.59, 85.52,
        119.42, 134.07, 121.27, 97.11, 73.20, 111.02, 79.75, 126.82
    ],
    'diesel': [
        409.71, 140.18, 531.52, 535.71, 692.92, 428.50, 1076.46, 360.07, 79.63, 1184.84,
        156.30, 66.15, 110.04, 100.48, 129.86, 822.28, 1230.77, 134.59, 146.69, 379.26,
        273.24, 135.82, 409.97, 651.66, 2488.36, 63.86, 51.34, 692.55, 532.86, 1453.16,
        85.99, 501.88, 389.98, 72.00, 56.09, 860.44, 1501.67, 218.89
    ],
    'unleaded': [
        326.09, 346.26, 240.61, 203.86, 309.11, 502.14, 409.49, 241.49, 158.58, 195.73,
        322.54, 226.82, 319.54, 390.34, 666.56, 565.47, 486.86, 608.04, 618.65, 610.26,
        155.84, 438.88, 203.03, 622.42, 315.71, 238.66, 167.12, 802.14, 214.58, 258.32,
        387.47, 433.76, 592.32, 276.60, 111.36, 322.46, 263.00, 374.16
    ]
}

# Calculate cumulative RMSE for each sales metric
for metric, rmse_values in sales_metrics.items():
    cumulative_rmse = np.sqrt(np.sum(np.square(rmse_values)))
    print(f"Cumulative RMSE for '{metric}' for all site IDs: {cumulative_rmse:.2f}")


Cumulative RMSE for 'daily_yoy_ndt.total_inside_sales' for all site IDs: 2374.15
Cumulative RMSE for 'daily_yoy_ndt.total_food_service' for all site IDs: 685.42
Cumulative RMSE for 'diesel' for all site IDs: 4447.96
Cumulative RMSE for 'unleaded' for all site IDs: 2485.20


In [ ]:
# Store feature importance for each metric and site ID
all_feature_importance = {}

for metric in sales_metrics:
    site_feature_importance = {}

    for site_id in timeseries['site_id_msba'].unique():
        try:
            # Filter data for a specific site
            site_data = timeseries[timeseries['site_id_msba'] == site_id]

            # Merge site-specific features from store_features
            site_features = store_features[store_features['site_id_msba'] == site_id]
            site_data = pd.merge(site_data, site_features, on='site_id_msba', how='left')

            # Features and target variable
            X = site_data.drop(columns=[metric])  # All columns except the target
            y = site_data[metric]

            # Drop non-numeric columns
            non_numeric_cols = X.select_dtypes(exclude=['int', 'float']).columns
            X = X.drop(columns=non_numeric_cols)

            # Store column names for later reference
            feature_columns = X.columns.tolist()

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

            # Initialize and fit XGBoost model
            model = XGBRegressor(n_estimators=100, learning_rate=0.1)  # Modify hyperparameters
            model.fit(X_train, y_train)

            # Capture feature importance for the current site and metric
            site_feature_importance[site_id] = model.feature_importances_

        except Exception as e:
            print(f"Error occurred for Site ID: {site_id} - Metric: {metric}. Details: {str(e)}")

    # Store feature importance for this metric across all sites
    all_feature_importance[metric] = site_feature_importance

# Display feature importance scores with feature names (excluding 'Unnamed: 0_x')
for metric, site_data in all_feature_importance.items():
    print(f"Metric: {metric}")
    for site_id, importance_scores in site_data.items():
        print(f"Site ID: {site_id}")
        for idx, importance_score in enumerate(importance_scores):
            if importance_score != 0 and feature_columns[idx] != 'Unnamed: 0_x':
                print(f"Feature {feature_columns[idx]}: Importance Score = {importance_score:.4f}")
        print("------------------------")
    print("========================")



Metric: daily_yoy_ndt.total_inside_sales
Site ID: 21560
Feature calendar.fiscal_week_id_for_year: Importance Score = 0.0147
Feature daily_yoy_ndt.total_inside_sales: Importance Score = 0.1324
Feature daily_yoy_ndt.total_food_service: Importance Score = 0.0044
Feature diesel: Importance Score = 0.7844
Feature Month: Importance Score = 0.0243
Feature Day: Importance Score = 0.0061
Feature DayOfYear: Importance Score = 0.0281
------------------------
Site ID: 22015
Feature calendar.fiscal_week_id_for_year: Importance Score = 0.0174
Feature daily_yoy_ndt.total_inside_sales: Importance Score = 0.0658
Feature daily_yoy_ndt.total_food_service: Importance Score = 0.0055
Feature diesel: Importance Score = 0.1034
Feature Month: Importance Score = 0.0637
Feature Day: Importance Score = 0.0056
Feature DayOfYear: Importance Score = 0.7341
------------------------
Site ID: 22085
Feature calendar.fiscal_week_id_for_year: Importance Score = 0.0025
Feature daily_yoy_ndt.total_inside_sales: Importance S

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

feature_importance_food_service = {}

metric = 'daily_yoy_ndt.total_food_service'  # Selected metric

for site_id in timeseries['site_id_msba'].unique():
    try:
        # Filter data for a specific site
        site_data = timeseries[timeseries['site_id_msba'] == site_id]

        # Merge site-specific features from store_features
        site_features = store_features[store_features['site_id_msba'] == site_id]
        site_data = pd.merge(site_data, site_features, on='site_id_msba', how='left')

        # Features and target variable
        X = site_data.drop(columns=[metric])  # All columns except the target
        y = site_data[metric]

        # Drop non-numeric columns
        non_numeric_cols = X.select_dtypes(exclude=['int', 'float']).columns
        X = X.drop(columns=non_numeric_cols)

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

        # Initialize and fit XGBoost model
        model = XGBRegressor(n_estimators=100, learning_rate=0.1)  # Modify hyperparameters
        model.fit(X_train, y_train)

        # Capture feature importance for the current site and metric
        feature_importance = model.feature_importances_

        # Store feature importance for this site
        feature_importance_food_service[site_id] = {
            feature: importance_score
            for feature, importance_score in zip(X.columns, feature_importance)
            if importance_score != 0 and feature != 'Unnamed: 0_x'
        }

    except Exception as e:
        print(f"Error occurred for Site ID: {site_id}. Details: {str(e)}")

# Display feature importance scores with feature names
print(f"Metric: {metric}")
for site_id, feature_scores in feature_importance_food_service.items():
    print(f"Site ID: {site_id}")
    for feature, importance_score in feature_scores.items():
        print(f"Feature {feature}: Importance Score = {importance_score:.4f}")
    print("------------------------")
print("========================")


Metric: daily_yoy_ndt.total_food_service
Site ID: 21560
Feature calendar.fiscal_week_id_for_year: Importance Score = 0.0408
Feature daily_yoy_ndt.total_inside_sales: Importance Score = 0.6951
Feature diesel: Importance Score = 0.2025
Feature unleaded: Importance Score = 0.0109
Feature Day: Importance Score = 0.0174
Feature DayOfYear: Importance Score = 0.0224
------------------------
Site ID: 22015
Feature calendar.fiscal_week_id_for_year: Importance Score = 0.0512
Feature daily_yoy_ndt.total_inside_sales: Importance Score = 0.1392
Feature diesel: Importance Score = 0.5985
Feature unleaded: Importance Score = 0.0301
Feature Month: Importance Score = 0.0994
Feature Day: Importance Score = 0.0248
Feature DayOfYear: Importance Score = 0.0471
------------------------
Site ID: 22085
Feature calendar.fiscal_week_id_for_year: Importance Score = 0.0118
Feature daily_yoy_ndt.total_inside_sales: Importance Score = 0.5908
Feature diesel: Importance Score = 0.0021
Feature unleaded: Importance Scor